In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
pd.options.display.max_columns=1000

In [2]:
#Picking 1 of the Lego tables that has a list of all sets by set_num and theme_id. Just to try the regression.
sets = pd.read_csv('C:/Users/marki/Desktop/SB Data Science BC/Springboard/Capstone2/csvs/sets.csv')
sets.head()

,set_num,name,year,theme_id,num_parts
0,001-1,Gears,1965,1,43
1,0011-2,Town Mini-Figures,1979,67,12
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,0
3,0012-1,Space Mini-Figures,1979,143,12
4,0013-1,Space Mini-Figures,1979,143,12


In [3]:
sets[sets['theme_id']==435].count()

set_num      417
name         417
year         417
theme_id     417
num_parts    417
dtype: int64

In [4]:
sets[sets['theme_id']!=435].count()

set_num      18090
name         18090
year         18090
theme_id     18090
num_parts    18090
dtype: int64

In [5]:
sets['set_num'].str.len()

0         5
1         6
2         6
3         6
4         6
         ..
18502     7
18503     7
18504    11
18505    10
18506     8
Name: set_num, Length: 18507, dtype: int64

In [6]:
sets['set_num_len'] = sets['set_num'].str.len()
sets.head()

,set_num,name,year,theme_id,num_parts,set_num_len
0,001-1,Gears,1965,1,43,5
1,0011-2,Town Mini-Figures,1979,67,12,6
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,0,6
3,0012-1,Space Mini-Figures,1979,143,12,6
4,0013-1,Space Mini-Figures,1979,143,12,6


In [7]:
sets['name'].str.len()

0         5
1        17
2        26
3        18
4        18
         ..
18502    19
18503    17
18504    31
18505    17
18506    20
Name: name, Length: 18507, dtype: int64

In [8]:
sets['name_len'] = sets['name'].str.len()
sets.head()

,set_num,name,year,theme_id,num_parts,set_num_len,name_len
0,001-1,Gears,1965,1,43,5,5
1,0011-2,Town Mini-Figures,1979,67,12,6,17
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,0,6,26
3,0012-1,Space Mini-Figures,1979,143,12,6,18
4,0013-1,Space Mini-Figures,1979,143,12,6,18


In [9]:
#Bring in the themes table so I can put a theme name to the set numbers 
themes = pd.read_csv('C:/Users/marki/Desktop/SB Data Science BC/Springboard/Capstone2/csvs/themes.csv')
themes.head()

,id,name,parent_id
0,1,Technic,NaN
1,3,Competition,1.0
2,4,Expert Builder,1.0
3,16,RoboRiders,1.0
4,17,Speed Slammers,1.0


In [10]:
themes.shape

(444, 3)

In [11]:
themes = themes.rename(columns={'id':'theme_id', 'name':'theme_name'})
themes.head()

,theme_id,theme_name,parent_id
0,1,Technic,NaN
1,3,Competition,1.0
2,4,Expert Builder,1.0
3,16,RoboRiders,1.0
4,17,Speed Slammers,1.0


In [12]:
#parent_id column is not needed
themes = themes.drop('parent_id', axis=1)
themes.head()

,theme_id,theme_name
0,1,Technic
1,3,Competition
2,4,Expert Builder
3,16,RoboRiders
4,17,Speed Slammers


In [13]:
df = sets.merge(themes, on='theme_id', how='left')
df.head()

,set_num,name,year,theme_id,num_parts,set_num_len,name_len,theme_name
0,001-1,Gears,1965,1,43,5,5,Technic
1,0011-2,Town Mini-Figures,1979,67,12,6,17,Classic Town
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,0,6,26,Lion Knights
3,0012-1,Space Mini-Figures,1979,143,12,6,18,Supplemental
4,0013-1,Space Mini-Figures,1979,143,12,6,18,Supplemental


In [14]:
df.shape

(18507, 8)

In [15]:
# List of the most used themes
df['theme_id'].value_counts()

501    1898
158     800
503     579
494     462
324     420
       ... 
34        1
156       1
241       1
307       1
512       1
Name: theme_id, Length: 435, dtype: int64

In [16]:
# Gear as a theme doesn't really help identify the set - gears can be used in many sets
themes[themes['theme_id'] == 501]

,theme_id,theme_name
255,501,Gear


In [17]:
# Supplemental as a theme also doesn't help identify a set.
themes[themes['theme_id'] == 158]

,theme_id,theme_name
78,158,Star Wars


In [18]:
# Duplo is quite different from regular Lego bricks and a duplo brick is easily identifiable to a duplo set.
themes[themes['theme_id'] == 503]

,theme_id,theme_name
257,503,Key Chain


In [19]:
# City as a theme is rich with many sets. I think this is the one I should use.
themes[themes['theme_id'] == 494]

,theme_id,theme_name
251,494,Friends


In [20]:
inventories = pd.read_csv('C:/Users/marki/Desktop/SB Data Science BC/Springboard/Capstone2/csvs/inventories.csv')
inventories.head()

,id,version,set_num
0,1,1,7922-1
1,3,1,3931-1
2,4,1,6942-1
3,15,1,5158-1
4,16,1,903-1


In [21]:
inventories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31277 entries, 0 to 31276
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       31277 non-null  int64 
 1   version  31277 non-null  int64 
 2   set_num  31277 non-null  object
dtypes: int64(2), object(1)
memory usage: 733.2+ KB


In [22]:
inventories.shape

(31277, 3)

In [23]:
df.shape

(18507, 8)

In [24]:
df = df.merge(inventories, on='set_num', how='left')
df.head()

,set_num,name,year,theme_id,num_parts,set_num_len,name_len,theme_name,id,version
0,001-1,Gears,1965,1,43,5,5,Technic,24696,1
1,0011-2,Town Mini-Figures,1979,67,12,6,17,Classic Town,5087,1
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,0,6,26,Lion Knights,2216,1
3,0012-1,Space Mini-Figures,1979,143,12,6,18,Supplemental,1414,1
4,0013-1,Space Mini-Figures,1979,143,12,6,18,Supplemental,4609,1


In [25]:
df.shape

(19675, 10)

In [26]:
df = df.drop('version', axis=1)
df.head()

,set_num,name,year,theme_id,num_parts,set_num_len,name_len,theme_name,id
0,001-1,Gears,1965,1,43,5,5,Technic,24696
1,0011-2,Town Mini-Figures,1979,67,12,6,17,Classic Town,5087
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,0,6,26,Lion Knights,2216
3,0012-1,Space Mini-Figures,1979,143,12,6,18,Supplemental,1414
4,0013-1,Space Mini-Figures,1979,143,12,6,18,Supplemental,4609


In [27]:
inventory_parts = pd.read_csv('C:/Users/marki/Desktop/SB Data Science BC/Springboard/Capstone2/csvs/inventory_parts.csv')
inventory_parts.head()

,inventory_id,part_num,color_id,quantity,is_spare
0,1,48379c01,72,1,f
1,1,48395,7,1,f
2,1,stickerupn0077,9999,1,f
3,1,upn0342,0,1,f
4,1,upn0350,25,1,f


In [28]:
inventory_parts = inventory_parts.rename(columns={'inventory_id':'id'})
inventory_parts.head()

,id,part_num,color_id,quantity,is_spare
0,1,48379c01,72,1,f
1,1,48395,7,1,f
2,1,stickerupn0077,9999,1,f
3,1,upn0342,0,1,f
4,1,upn0350,25,1,f


In [29]:
inventory_parts.shape

(972396, 5)

In [30]:
colors = pd.read_csv('C:/Users/marki/Desktop/SB Data Science BC/Springboard/Capstone2/csvs/colors.csv')
colors.head()

,id,name,rgb,is_trans
0,-1,[Unknown],0033B2,f
1,0,Black,05131D,f
2,1,Blue,0055BF,f
3,2,Green,237841,f
4,3,Dark Turquoise,008F9B,f


In [31]:
colors = colors.rename(columns={'id':'color_id', 'name':'color_name'})
colors.head()

,color_id,color_name,rgb,is_trans
0,-1,[Unknown],0033B2,f
1,0,Black,05131D,f
2,1,Blue,0055BF,f
3,2,Green,237841,f
4,3,Dark Turquoise,008F9B,f


In [32]:
colors.shape

(190, 4)

In [33]:
dfB = inventory_parts.merge(colors, on='color_id', how='left')
dfB.head()

,id,part_num,color_id,quantity,is_spare,color_name,rgb,is_trans
0,1,48379c01,72,1,f,Dark Bluish Gray,6C6E68,f
1,1,48395,7,1,f,Light Gray,9BA19D,f
2,1,stickerupn0077,9999,1,f,[No Color/Any Color],05131D,f
3,1,upn0342,0,1,f,Black,05131D,f
4,1,upn0350,25,1,f,Orange,FE8A18,f


In [34]:
dfB.shape

(972396, 8)

In [35]:
dfB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 972396 entries, 0 to 972395
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          972396 non-null  int64 
 1   part_num    972396 non-null  object
 2   color_id    972396 non-null  int64 
 3   quantity    972396 non-null  int64 
 4   is_spare    972396 non-null  object
 5   color_name  972396 non-null  object
 6   rgb         972396 non-null  object
 7   is_trans    972396 non-null  object
dtypes: int64(3), object(5)
memory usage: 66.8+ MB


In [36]:
parts = pd.read_csv('C:/Users/marki/Desktop/SB Data Science BC/Springboard/Capstone2/csvs/parts.csv')
parts.head()

,part_num,name,part_cat_id,part_material
0,003381,Sticker Sheet for Set 663-1,58,Plastic
1,003383,"Sticker Sheet for Sets 618-1, 628-2",58,Plastic
2,003402,"Sticker Sheet for Sets 310-3, 311-1, 312-3",58,Plastic
3,003429,Sticker Sheet for Set 1550-1,58,Plastic
4,003432,"Sticker Sheet for Sets 357-1, 355-1, 940-1",58,Plastic


In [37]:
parts = parts.rename(columns={'name':'part_name'})
parts.head()

,part_num,part_name,part_cat_id,part_material
0,003381,Sticker Sheet for Set 663-1,58,Plastic
1,003383,"Sticker Sheet for Sets 618-1, 628-2",58,Plastic
2,003402,"Sticker Sheet for Sets 310-3, 311-1, 312-3",58,Plastic
3,003429,Sticker Sheet for Set 1550-1,58,Plastic
4,003432,"Sticker Sheet for Sets 357-1, 355-1, 940-1",58,Plastic


In [38]:
parts.shape

(44074, 4)

In [39]:
dfB = dfB.merge(parts, on='part_num', how='left')
dfB.head()

,id,part_num,color_id,quantity,is_spare,color_name,rgb,is_trans,part_name,part_cat_id,part_material
0,1,48379c01,72,1,f,Dark Bluish Gray,6C6E68,f,"Large Figure Torso and Legs, Promo Figure Base...",41,Plastic
1,1,48395,7,1,f,Light Gray,9BA19D,f,Sports Snowboard from McDonald's Promotional Set,27,Plastic
2,1,stickerupn0077,9999,1,f,[No Color/Any Color],05131D,f,Sticker Sheet for Set 7922-1,58,Plastic
3,1,upn0342,0,1,f,Black,05131D,f,Sports Promo Paddle from McDonald's Sports Sets,27,Plastic
4,1,upn0350,25,1,f,Orange,FE8A18,f,Sports Promo Figure Head Torso Assembly McDona...,13,Plastic


In [40]:
dfB.shape

(972396, 11)

In [41]:
dfB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 972396 entries, 0 to 972395
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             972396 non-null  int64 
 1   part_num       972396 non-null  object
 2   color_id       972396 non-null  int64 
 3   quantity       972396 non-null  int64 
 4   is_spare       972396 non-null  object
 5   color_name     972396 non-null  object
 6   rgb            972396 non-null  object
 7   is_trans       972396 non-null  object
 8   part_name      972396 non-null  object
 9   part_cat_id    972396 non-null  int64 
 10  part_material  972396 non-null  object
dtypes: int64(4), object(7)
memory usage: 89.0+ MB


In [42]:
dfB = dfB.drop('is_spare', axis=1)

In [43]:
dfB = dfB.drop('rgb', axis=1)

In [44]:
dfB = dfB.drop('part_cat_id', axis=1)
dfB.head()

,id,part_num,color_id,quantity,color_name,is_trans,part_name,part_material
0,1,48379c01,72,1,Dark Bluish Gray,f,"Large Figure Torso and Legs, Promo Figure Base...",Plastic
1,1,48395,7,1,Light Gray,f,Sports Snowboard from McDonald's Promotional Set,Plastic
2,1,stickerupn0077,9999,1,[No Color/Any Color],f,Sticker Sheet for Set 7922-1,Plastic
3,1,upn0342,0,1,Black,f,Sports Promo Paddle from McDonald's Sports Sets,Plastic
4,1,upn0350,25,1,Orange,f,Sports Promo Figure Head Torso Assembly McDona...,Plastic


In [45]:
dfB = pd.get_dummies(dfB, columns=['is_trans'])
dfB.head()

,id,part_num,color_id,quantity,color_name,part_name,part_material,is_trans_f,is_trans_t
0,1,48379c01,72,1,Dark Bluish Gray,"Large Figure Torso and Legs, Promo Figure Base...",Plastic,1,0
1,1,48395,7,1,Light Gray,Sports Snowboard from McDonald's Promotional Set,Plastic,1,0
2,1,stickerupn0077,9999,1,[No Color/Any Color],Sticker Sheet for Set 7922-1,Plastic,1,0
3,1,upn0342,0,1,Black,Sports Promo Paddle from McDonald's Sports Sets,Plastic,1,0
4,1,upn0350,25,1,Orange,Sports Promo Figure Head Torso Assembly McDona...,Plastic,1,0


In [46]:
dfB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 972396 entries, 0 to 972395
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             972396 non-null  int64 
 1   part_num       972396 non-null  object
 2   color_id       972396 non-null  int64 
 3   quantity       972396 non-null  int64 
 4   color_name     972396 non-null  object
 5   part_name      972396 non-null  object
 6   part_material  972396 non-null  object
 7   is_trans_f     972396 non-null  uint8 
 8   is_trans_t     972396 non-null  uint8 
dtypes: int64(3), object(4), uint8(2)
memory usage: 61.2+ MB


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19675 entries, 0 to 19674
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   set_num      19675 non-null  object
 1   name         19675 non-null  object
 2   year         19675 non-null  int64 
 3   theme_id     19675 non-null  int64 
 4   num_parts    19675 non-null  int64 
 5   set_num_len  19675 non-null  int64 
 6   name_len     19675 non-null  int64 
 7   theme_name   19675 non-null  object
 8   id           19675 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 1.5+ MB


In [48]:
dfB['part_name'].isnull().sum()

0

In [49]:
dfB[dfB.isnull().any(axis=1)].head()

,id,part_num,color_id,quantity,color_name,part_name,part_material,is_trans_f,is_trans_t


In [50]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19675 entries, 0 to 19674
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   set_num      19675 non-null  object
 1   name         19675 non-null  object
 2   year         19675 non-null  int64 
 3   theme_id     19675 non-null  int64 
 4   num_parts    19675 non-null  int64 
 5   set_num_len  19675 non-null  int64 
 6   name_len     19675 non-null  int64 
 7   theme_name   19675 non-null  object
 8   id           19675 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 1.5+ MB


In [51]:
dfB = dfB.fillna('None')
dfB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 972396 entries, 0 to 972395
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             972396 non-null  int64 
 1   part_num       972396 non-null  object
 2   color_id       972396 non-null  int64 
 3   quantity       972396 non-null  int64 
 4   color_name     972396 non-null  object
 5   part_name      972396 non-null  object
 6   part_material  972396 non-null  object
 7   is_trans_f     972396 non-null  uint8 
 8   is_trans_t     972396 non-null  uint8 
dtypes: int64(3), object(4), uint8(2)
memory usage: 61.2+ MB


In [52]:
dfB[dfB['part_name'] == 'None']

,id,part_num,color_id,quantity,color_name,part_name,part_material,is_trans_f,is_trans_t


In [53]:
df = df.merge(dfB, on='id', how='inner')
df.head()

,set_num,name,year,theme_id,num_parts,set_num_len,name_len,theme_name,id,part_num,color_id,quantity,color_name,part_name,part_material,is_trans_f,is_trans_t
0,001-1,Gears,1965,1,43,5,5,Technic,24696,132a,7,4,Light Gray,Tyre Smooth Old Style - Small,Rubber,1,0
1,001-1,Gears,1965,1,43,5,5,Technic,24696,3020,15,4,White,Plate 2 x 4,Plastic,1,0
2,001-1,Gears,1965,1,43,5,5,Technic,24696,3062c,15,1,White,"Brick Round 1 x 1 Solid Stud, No Bottom Groove",Plastic,1,0
3,001-1,Gears,1965,1,43,5,5,Technic,24696,3404bc01,15,4,White,"Turntable 4 x 4 - Old Type Complete, Perfectly...",Plastic,1,0
4,001-1,Gears,1965,1,43,5,5,Technic,24696,36,7,4,Light Gray,Tyre Smooth Old Style - Large,Rubber,1,0


In [54]:
df.shape

(919288, 17)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 919288 entries, 0 to 919287
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   set_num        919288 non-null  object
 1   name           919288 non-null  object
 2   year           919288 non-null  int64 
 3   theme_id       919288 non-null  int64 
 4   num_parts      919288 non-null  int64 
 5   set_num_len    919288 non-null  int64 
 6   name_len       919288 non-null  int64 
 7   theme_name     919288 non-null  object
 8   id             919288 non-null  int64 
 9   part_num       919288 non-null  object
 10  color_id       919288 non-null  int64 
 11  quantity       919288 non-null  int64 
 12  color_name     919288 non-null  object
 13  part_name      919288 non-null  object
 14  part_material  919288 non-null  object
 15  is_trans_f     919288 non-null  uint8 
 16  is_trans_t     919288 non-null  uint8 
dtypes: int64(8), object(7), uint8(2)
memory usage: 1

In [56]:
df = df.drop('is_trans_f', axis=1)
df.head()

,set_num,name,year,theme_id,num_parts,set_num_len,name_len,theme_name,id,part_num,color_id,quantity,color_name,part_name,part_material,is_trans_t
0,001-1,Gears,1965,1,43,5,5,Technic,24696,132a,7,4,Light Gray,Tyre Smooth Old Style - Small,Rubber,0
1,001-1,Gears,1965,1,43,5,5,Technic,24696,3020,15,4,White,Plate 2 x 4,Plastic,0
2,001-1,Gears,1965,1,43,5,5,Technic,24696,3062c,15,1,White,"Brick Round 1 x 1 Solid Stud, No Bottom Groove",Plastic,0
3,001-1,Gears,1965,1,43,5,5,Technic,24696,3404bc01,15,4,White,"Turntable 4 x 4 - Old Type Complete, Perfectly...",Plastic,0
4,001-1,Gears,1965,1,43,5,5,Technic,24696,36,7,4,Light Gray,Tyre Smooth Old Style - Large,Rubber,0


In [57]:
df = df.drop('part_material', axis=1)
df.head()

,set_num,name,year,theme_id,num_parts,set_num_len,name_len,theme_name,id,part_num,color_id,quantity,color_name,part_name,is_trans_t
0,001-1,Gears,1965,1,43,5,5,Technic,24696,132a,7,4,Light Gray,Tyre Smooth Old Style - Small,0
1,001-1,Gears,1965,1,43,5,5,Technic,24696,3020,15,4,White,Plate 2 x 4,0
2,001-1,Gears,1965,1,43,5,5,Technic,24696,3062c,15,1,White,"Brick Round 1 x 1 Solid Stud, No Bottom Groove",0
3,001-1,Gears,1965,1,43,5,5,Technic,24696,3404bc01,15,4,White,"Turntable 4 x 4 - Old Type Complete, Perfectly...",0
4,001-1,Gears,1965,1,43,5,5,Technic,24696,36,7,4,Light Gray,Tyre Smooth Old Style - Large,0


In [58]:
df = df.rename(columns={'name':'set_name'})
df.head()

,set_num,set_name,year,theme_id,num_parts,set_num_len,name_len,theme_name,id,part_num,color_id,quantity,color_name,part_name,is_trans_t
0,001-1,Gears,1965,1,43,5,5,Technic,24696,132a,7,4,Light Gray,Tyre Smooth Old Style - Small,0
1,001-1,Gears,1965,1,43,5,5,Technic,24696,3020,15,4,White,Plate 2 x 4,0
2,001-1,Gears,1965,1,43,5,5,Technic,24696,3062c,15,1,White,"Brick Round 1 x 1 Solid Stud, No Bottom Groove",0
3,001-1,Gears,1965,1,43,5,5,Technic,24696,3404bc01,15,4,White,"Turntable 4 x 4 - Old Type Complete, Perfectly...",0
4,001-1,Gears,1965,1,43,5,5,Technic,24696,36,7,4,Light Gray,Tyre Smooth Old Style - Large,0


In [59]:
df = df.rename(columns={'name_len':'set_name_len'})
df.head()

,set_num,set_name,year,theme_id,num_parts,set_num_len,set_name_len,theme_name,id,part_num,color_id,quantity,color_name,part_name,is_trans_t
0,001-1,Gears,1965,1,43,5,5,Technic,24696,132a,7,4,Light Gray,Tyre Smooth Old Style - Small,0
1,001-1,Gears,1965,1,43,5,5,Technic,24696,3020,15,4,White,Plate 2 x 4,0
2,001-1,Gears,1965,1,43,5,5,Technic,24696,3062c,15,1,White,"Brick Round 1 x 1 Solid Stud, No Bottom Groove",0
3,001-1,Gears,1965,1,43,5,5,Technic,24696,3404bc01,15,4,White,"Turntable 4 x 4 - Old Type Complete, Perfectly...",0
4,001-1,Gears,1965,1,43,5,5,Technic,24696,36,7,4,Light Gray,Tyre Smooth Old Style - Large,0


In [60]:
df['theme_name_len'] = df['theme_name'].str.len()
df.head()

,set_num,set_name,year,theme_id,num_parts,set_num_len,set_name_len,theme_name,id,part_num,color_id,quantity,color_name,part_name,is_trans_t,theme_name_len
0,001-1,Gears,1965,1,43,5,5,Technic,24696,132a,7,4,Light Gray,Tyre Smooth Old Style - Small,0,7
1,001-1,Gears,1965,1,43,5,5,Technic,24696,3020,15,4,White,Plate 2 x 4,0,7
2,001-1,Gears,1965,1,43,5,5,Technic,24696,3062c,15,1,White,"Brick Round 1 x 1 Solid Stud, No Bottom Groove",0,7
3,001-1,Gears,1965,1,43,5,5,Technic,24696,3404bc01,15,4,White,"Turntable 4 x 4 - Old Type Complete, Perfectly...",0,7
4,001-1,Gears,1965,1,43,5,5,Technic,24696,36,7,4,Light Gray,Tyre Smooth Old Style - Large,0,7


In [61]:
df['part_num_len'] = df['part_num'].str.len()
df.head()

,set_num,set_name,year,theme_id,num_parts,set_num_len,set_name_len,theme_name,id,part_num,color_id,quantity,color_name,part_name,is_trans_t,theme_name_len,part_num_len
0,001-1,Gears,1965,1,43,5,5,Technic,24696,132a,7,4,Light Gray,Tyre Smooth Old Style - Small,0,7,4
1,001-1,Gears,1965,1,43,5,5,Technic,24696,3020,15,4,White,Plate 2 x 4,0,7,4
2,001-1,Gears,1965,1,43,5,5,Technic,24696,3062c,15,1,White,"Brick Round 1 x 1 Solid Stud, No Bottom Groove",0,7,5
3,001-1,Gears,1965,1,43,5,5,Technic,24696,3404bc01,15,4,White,"Turntable 4 x 4 - Old Type Complete, Perfectly...",0,7,8
4,001-1,Gears,1965,1,43,5,5,Technic,24696,36,7,4,Light Gray,Tyre Smooth Old Style - Large,0,7,2


In [62]:
df['color_name_len'] = df['color_name'].str.len()
df.head()

,set_num,set_name,year,theme_id,num_parts,set_num_len,set_name_len,theme_name,id,part_num,color_id,quantity,color_name,part_name,is_trans_t,theme_name_len,part_num_len,color_name_len
0,001-1,Gears,1965,1,43,5,5,Technic,24696,132a,7,4,Light Gray,Tyre Smooth Old Style - Small,0,7,4,10
1,001-1,Gears,1965,1,43,5,5,Technic,24696,3020,15,4,White,Plate 2 x 4,0,7,4,5
2,001-1,Gears,1965,1,43,5,5,Technic,24696,3062c,15,1,White,"Brick Round 1 x 1 Solid Stud, No Bottom Groove",0,7,5,5
3,001-1,Gears,1965,1,43,5,5,Technic,24696,3404bc01,15,4,White,"Turntable 4 x 4 - Old Type Complete, Perfectly...",0,7,8,5
4,001-1,Gears,1965,1,43,5,5,Technic,24696,36,7,4,Light Gray,Tyre Smooth Old Style - Large,0,7,2,10


In [63]:
df['part_name_len'] = df['part_name'].str.len()
df.head()

,set_num,set_name,year,theme_id,num_parts,set_num_len,set_name_len,theme_name,id,part_num,color_id,quantity,color_name,part_name,is_trans_t,theme_name_len,part_num_len,color_name_len,part_name_len
0,001-1,Gears,1965,1,43,5,5,Technic,24696,132a,7,4,Light Gray,Tyre Smooth Old Style - Small,0,7,4,10,29
1,001-1,Gears,1965,1,43,5,5,Technic,24696,3020,15,4,White,Plate 2 x 4,0,7,4,5,11
2,001-1,Gears,1965,1,43,5,5,Technic,24696,3062c,15,1,White,"Brick Round 1 x 1 Solid Stud, No Bottom Groove",0,7,5,5,46
3,001-1,Gears,1965,1,43,5,5,Technic,24696,3404bc01,15,4,White,"Turntable 4 x 4 - Old Type Complete, Perfectly...",0,7,8,5,52
4,001-1,Gears,1965,1,43,5,5,Technic,24696,36,7,4,Light Gray,Tyre Smooth Old Style - Large,0,7,2,10,29


In [64]:
df = df.rename(columns={'is_trans_t':'is_trans'})
df.head(3)

,set_num,set_name,year,theme_id,num_parts,set_num_len,set_name_len,theme_name,id,part_num,color_id,quantity,color_name,part_name,is_trans,theme_name_len,part_num_len,color_name_len,part_name_len
0,001-1,Gears,1965,1,43,5,5,Technic,24696,132a,7,4,Light Gray,Tyre Smooth Old Style - Small,0,7,4,10,29
1,001-1,Gears,1965,1,43,5,5,Technic,24696,3020,15,4,White,Plate 2 x 4,0,7,4,5,11
2,001-1,Gears,1965,1,43,5,5,Technic,24696,3062c,15,1,White,"Brick Round 1 x 1 Solid Stud, No Bottom Groove",0,7,5,5,46


In [65]:
is_train = np.random.rand(df.shape[0]) > 0.1
is_train.sum() / df.shape[0]

0.9000976842948021

In [66]:
df['theme_id'].value_counts()

158    67154
494    45129
1      38592
435    35468
672    23836
       ...  
307       11
369       11
368        2
559        1
241        1
Name: theme_id, Length: 432, dtype: int64

In [67]:
themes[themes['theme_id'] == 158]

,theme_id,theme_name
78,158,Star Wars


In [68]:
themes[themes['theme_id'] == 494]

,theme_id,theme_name
251,494,Friends


In [69]:
themes[themes['theme_id'] == 1]

,theme_id,theme_name
0,1,Technic


In [70]:
themes[themes['theme_id'] == 435]

,theme_id,theme_name
219,435,Ninjago


In [71]:
df['is_ninjago'] = df['theme_id'] == 435
df.head()

,set_num,set_name,year,theme_id,num_parts,set_num_len,set_name_len,theme_name,id,part_num,color_id,quantity,color_name,part_name,is_trans,theme_name_len,part_num_len,color_name_len,part_name_len,is_ninjago
0,001-1,Gears,1965,1,43,5,5,Technic,24696,132a,7,4,Light Gray,Tyre Smooth Old Style - Small,0,7,4,10,29,False
1,001-1,Gears,1965,1,43,5,5,Technic,24696,3020,15,4,White,Plate 2 x 4,0,7,4,5,11,False
2,001-1,Gears,1965,1,43,5,5,Technic,24696,3062c,15,1,White,"Brick Round 1 x 1 Solid Stud, No Bottom Groove",0,7,5,5,46,False
3,001-1,Gears,1965,1,43,5,5,Technic,24696,3404bc01,15,4,White,"Turntable 4 x 4 - Old Type Complete, Perfectly...",0,7,8,5,52,False
4,001-1,Gears,1965,1,43,5,5,Technic,24696,36,7,4,Light Gray,Tyre Smooth Old Style - Large,0,7,2,10,29,False


In [72]:
df[df['theme_id'] == 435]

,set_num,set_name,year,theme_id,num_parts,set_num_len,set_name_len,theme_name,id,part_num,color_id,quantity,color_name,part_name,is_trans,theme_name_len,part_num_len,color_name_len,part_name_len,is_ninjago
74598,111901-1,Garmadon,2019,435,10,8,8,Ninjago,34764,21459,179,4,Flat Silver,Weapon Sword / Katana / Shamshir with Capped P...,0,7,5,11,66,True
74599,111902-1,Wu,2019,435,6,8,2,Ninjago,39226,63965,297,1,Pearl Gold,Bar 6L with Stop Ring,0,7,5,10,21,True
74600,111903-1,Kai vs. Wyplash,2019,435,13,8,15,Ninjago,39213,21459,179,1,Flat Silver,Weapon Sword / Katana / Shamshir with Capped P...,0,7,5,11,66,True
74601,111903-1,Kai vs. Wyplash,2019,435,13,8,15,Ninjago,39213,60752,28,1,Dark Tan,Weapon Sword / Scimitar Notched Blade,0,7,5,8,37,True
74602,111904-1,Jay vs. Lasha,2019,435,12,8,13,Ninjago,42111,59229,70,1,Reddish Brown,Weapon Sword Scythe Blade with Clip Pommel,0,7,5,13,42,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918431,TRUNINJAGO-4,Micro Morro Dragon,2015,435,29,12,18,Ninjago,15797,54200,0,1,Black,Slope 30° 1 x 1 x 2/3 (Cheese Slope),0,7,5,5,36,True
918432,TRUNINJAGO-4,Micro Morro Dragon,2015,435,29,12,18,Ninjago,15797,54200,72,4,Dark Bluish Gray,Slope 30° 1 x 1 x 2/3 (Cheese Slope),0,7,5,16,36,True
918433,TRUNINJAGO-4,Micro Morro Dragon,2015,435,29,12,18,Ninjago,15797,59900,0,1,Black,Cone 1 x 1 [Top Groove],0,7,5,5,23,True
918434,TRUNINJAGO-4,Micro Morro Dragon,2015,435,29,12,18,Ninjago,15797,6133,0,2,Black,"Creature Body Part, Dragon Wing",0,7,4,5,31,True


In [73]:
df = pd.get_dummies(df, columns=['is_ninjago'])
df.head()

,set_num,set_name,year,theme_id,num_parts,set_num_len,set_name_len,theme_name,id,part_num,color_id,quantity,color_name,part_name,is_trans,theme_name_len,part_num_len,color_name_len,part_name_len,is_ninjago_False,is_ninjago_True
0,001-1,Gears,1965,1,43,5,5,Technic,24696,132a,7,4,Light Gray,Tyre Smooth Old Style - Small,0,7,4,10,29,1,0
1,001-1,Gears,1965,1,43,5,5,Technic,24696,3020,15,4,White,Plate 2 x 4,0,7,4,5,11,1,0
2,001-1,Gears,1965,1,43,5,5,Technic,24696,3062c,15,1,White,"Brick Round 1 x 1 Solid Stud, No Bottom Groove",0,7,5,5,46,1,0
3,001-1,Gears,1965,1,43,5,5,Technic,24696,3404bc01,15,4,White,"Turntable 4 x 4 - Old Type Complete, Perfectly...",0,7,8,5,52,1,0
4,001-1,Gears,1965,1,43,5,5,Technic,24696,36,7,4,Light Gray,Tyre Smooth Old Style - Large,0,7,2,10,29,1,0


In [74]:
df = df.drop('is_ninjago_False', axis=1)
df.head()

,set_num,set_name,year,theme_id,num_parts,set_num_len,set_name_len,theme_name,id,part_num,color_id,quantity,color_name,part_name,is_trans,theme_name_len,part_num_len,color_name_len,part_name_len,is_ninjago_True
0,001-1,Gears,1965,1,43,5,5,Technic,24696,132a,7,4,Light Gray,Tyre Smooth Old Style - Small,0,7,4,10,29,0
1,001-1,Gears,1965,1,43,5,5,Technic,24696,3020,15,4,White,Plate 2 x 4,0,7,4,5,11,0
2,001-1,Gears,1965,1,43,5,5,Technic,24696,3062c,15,1,White,"Brick Round 1 x 1 Solid Stud, No Bottom Groove",0,7,5,5,46,0
3,001-1,Gears,1965,1,43,5,5,Technic,24696,3404bc01,15,4,White,"Turntable 4 x 4 - Old Type Complete, Perfectly...",0,7,8,5,52,0
4,001-1,Gears,1965,1,43,5,5,Technic,24696,36,7,4,Light Gray,Tyre Smooth Old Style - Large,0,7,2,10,29,0


In [75]:
df = df.rename(columns={'is_ninjago_True':'is_ninjago'})
df.head()

,set_num,set_name,year,theme_id,num_parts,set_num_len,set_name_len,theme_name,id,part_num,color_id,quantity,color_name,part_name,is_trans,theme_name_len,part_num_len,color_name_len,part_name_len,is_ninjago
0,001-1,Gears,1965,1,43,5,5,Technic,24696,132a,7,4,Light Gray,Tyre Smooth Old Style - Small,0,7,4,10,29,0
1,001-1,Gears,1965,1,43,5,5,Technic,24696,3020,15,4,White,Plate 2 x 4,0,7,4,5,11,0
2,001-1,Gears,1965,1,43,5,5,Technic,24696,3062c,15,1,White,"Brick Round 1 x 1 Solid Stud, No Bottom Groove",0,7,5,5,46,0
3,001-1,Gears,1965,1,43,5,5,Technic,24696,3404bc01,15,4,White,"Turntable 4 x 4 - Old Type Complete, Perfectly...",0,7,8,5,52,0
4,001-1,Gears,1965,1,43,5,5,Technic,24696,36,7,4,Light Gray,Tyre Smooth Old Style - Large,0,7,2,10,29,0


In [76]:
df.to_csv('C:/Users/marki/Desktop/SB Data Science BC/Springboard/Capstone2/data from notebooks/df.csv', index=False)

In [77]:
df.shape

(919288, 20)

In [78]:
is_ninjago = df['is_ninjago'].sum() / df.shape[0]
is_ninjago

0.03858203305166607